In [12]:
import os
import logging
from io import BytesIO
import time
import zipfile
import numpy as np
import boto3
from datetime import datetime, timezone
from time import gmtime, strftime
import json
import pandas as pd
import matplotlib.pyplot as plt
import pickle

In [157]:
import math

In [13]:
# The difference between UTC and local timezone
timezone_offset = 0

### Function Name List

In [14]:
function_prefix = "Structures"
function_count = 4
function_name_list = [function_prefix+'_f'+str(i) for i in range(1, function_count+1)]
print(function_name_list)

['Structures_f1', 'Structures_f2', 'Structures_f3', 'Structures_f4']


In [15]:
mem_config_list={
    'f1':1280,
    'f2':896,
    'f3':1536,
    'f4':1088
}

# Execute the Application

In [3]:
sfn_client = boto3.client('stepfunctions')

In [4]:
stateMachineArn='arn:aws:states:us-east-2:499537426559:stateMachine:Map'

The serverless application workflow can be found in the README.md.

## Test Run

In [8]:
sfn_client.start_execution(
    stateMachineArn=stateMachineArn,
    input="""{\"map_list\":["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]}"""
)

{'executionArn': 'arn:aws:states:us-east-2:499537426559:express:Map:2f148ec1-db3f-4c49-8941-a2102d6e96ee:ef398211-cdd0-4ce2-a12d-6130d976a46a',
 'startDate': datetime.datetime(2022, 1, 4, 20, 6, 3, 581000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '7f9b97f2-e682-4759-958b-39faf835e290',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7f9b97f2-e682-4759-958b-39faf835e290',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '171'},
  'RetryAttempts': 0}}

## Configure Logging

In [9]:
logging.basicConfig(filename='AppExecution.log', encoding='utf-8', format='%(asctime)s.%(msecs)03d %(message)s', datefmt='%Y-%m-%d %H:%M:%S', level=logging.INFO)

## Execute Map

In [10]:
np.random.seed(256)

In [ ]:
for i in range(5000):
    payload_str="""{\"map_list\":["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]}"""
    response = sfn_client.start_execution(stateMachineArn=stateMachineArn, input=payload_str)
    RequestId = response.get('ResponseMetadata', {}).get('RequestId')
    StatusCode = response.get('ResponseMetadata', {}).get('HTTPStatusCode', 'ERR')
    logging.info(f'{i+1} {StatusCode} {RequestId}')
    time.sleep(10)

## Get the start time and the end time

In [16]:
app_exeuction_start_time = ' '.join(os.popen('head -1 AppExecution.log').read().split(' ')[:2])
app_execution_end_time = ' '.join(os.popen('tail -1 AppExecution.log').read().split(' ')[:2])
app_exeuction_start_time = datetime.strptime(app_exeuction_start_time, '%Y-%m-%d %H:%M:%S.%f')
app_execution_end_time = datetime.strptime(app_execution_end_time, '%Y-%m-%d %H:%M:%S.%f')

In [17]:
app_exeuction_start_time

datetime.datetime(2022, 1, 4, 20, 7, 41, 988000)

In [18]:
app_execution_end_time

datetime.datetime(2022, 1, 5, 10, 13, 44, 352000)

In [19]:
app_exeuction_start_time = int(datetime.timestamp(app_exeuction_start_time))

In [20]:
app_execution_end_time = int(datetime.timestamp(app_execution_end_time)) + 20

# Retrieve Logs

In [21]:
logclient = boto3.client('logs')

In [40]:
timestamp_list = [' '.join(os.popen(f'head -1 AppExecution.log').read().split(' ')[:2])]
for i in range(1000, 6000, 1000):
    timestamp_list.append(' '.join(os.popen(f'head -{i} AppExecution.log | tail -1').read().split(' ')[:2]))

## Query Step Functions Logs

In [43]:
query_sfn_Map = logclient.start_query(
    logGroupName='/aws/vendedlogs/states/{}-Logs'.format('Map'),
    queryString="fields type, @timestamp| filter type = 'ExecutionStarted' or type = 'ExecutionSucceeded' | sort id desc",
    startTime=app_exeuction_start_time,
    endTime=app_execution_end_time,
    limit = 10000
)

In [44]:
query_results_sfn_Map = logclient.get_query_results(
    queryId=query_sfn_Map['queryId']
)

In [45]:
Map_starttimestamp = np.sort([datetime.timestamp(datetime.strptime(item[1]['value'], '%Y-%m-%d %H:%M:%S.%f'))+timezone_offset*3600 for item in query_results_sfn_Map['results'] if item[0]['value']=='ExecutionStarted'])
Map_endtimestamp = np.sort([datetime.timestamp(datetime.strptime(item[1]['value'], '%Y-%m-%d %H:%M:%S.%f'))+timezone_offset*3600 for item in query_results_sfn_Map['results'] if item[0]['value']=='ExecutionSucceeded'])

In [46]:
pd.DataFrame({'Start': Map_starttimestamp, 'End':Map_endtimestamp}).to_csv('Map_SFN_Logs.csv', index=False)


In [47]:
Map_sfn_logs = pd.read_csv('Map_SFN_Logs.csv', low_memory=False)

In [48]:
Map_sfn_logs.shape

(5000, 2)

## Query Lambda Function Logs

### Functions for parsing Logs

In [29]:
def lambda_report_log_to_dict(log):
    res={}
    lis=[item.split(': ') for item in log[1]['value'].split('\t')]
    res['RequestId']=lis[0][1]
    res['Duration']=float(lis[1][1].split(' ')[0])
    res['Billed_Duration']=int(lis[2][1].split(' ')[0])
    res['Memory_Size']=int(lis[3][1].split(' ')[0])
    res['Max_Memory_Used']=int(lis[4][1].split(' ')[0])
    res['UTC_Timestamp'] = time.mktime(datetime.strptime(log[0]['value'], "%Y-%m-%d %H:%M:%S.%f").timetuple()) +timezone_offset*3600
    return res

In [49]:
timestamp_list

['2022-01-04 20:07:41.988',
 '2022-01-04 22:56:45.211',
 '2022-01-05 01:45:59.467',
 '2022-01-05 04:35:14.979',
 '2022-01-05 07:24:26.929',
 '2022-01-05 10:13:44.352']

In [57]:
def process_time_string(s: str) -> int:
    t = datetime.strptime(s, '%Y-%m-%d %H:%M:%S.%f')
    t = datetime.timestamp(t)
    return int(t)

### Prepare and Retrieve Logs

In [139]:
Map_lambda_logs_dict = {'f'+str(i):[] for i in range(1, function_count+1)}

In [140]:
query_lambda = []
for function in function_name_list:
    query_lambda.append(logclient.start_query(
        logGroupName='/aws/lambda/{}'.format(function),
        queryString="fields @timestamp, @message| filter @message like 'REPORT'| sort @timestamp asc",
        startTime=process_time_string('2022-01-04 20:07:41.988') - 5,
        endTime=process_time_string('2022-01-04 22:56:45.211') - 5,
        limit=10000
        ))
    time.sleep(4)
time.sleep(10)
query_lambda_results = []
for q in query_lambda:
    query_lambda_results.append(logclient.get_query_results(
        queryId=q['queryId']
    ))
    time.sleep(4)
with open('query_lambda_results_part1.pickle', 'wb') as f:
    f.write(pickle.dumps(query_lambda_results))
for i in range(1, function_count+1):
    Map_lambda_logs_dict['f'+str(i)] = [lambda_report_log_to_dict(item) for item in query_lambda_results[i-1]['results']]
    for item in Map_lambda_logs_dict['f'+str(i)]:
        item['Function']='f'+str(i)

In [141]:
query_lambda = []
for function in function_name_list:
    query_lambda.append(logclient.start_query(
        logGroupName='/aws/lambda/{}'.format(function),
        queryString="fields @timestamp, @message| filter @message like 'REPORT'| sort @timestamp asc",
        startTime=process_time_string('2022-01-04 22:56:45.211') - 5,
        endTime=process_time_string('2022-01-05 01:45:59.467') - 5,
        limit=10000
        ))
    time.sleep(4)
time.sleep(10)
query_lambda_results = []
for q in query_lambda:
    query_lambda_results.append(logclient.get_query_results(
        queryId=q['queryId']
    ))
    time.sleep(4)
with open('query_lambda_results_part2.pickle', 'wb') as f:
    f.write(pickle.dumps(query_lambda_results))
for i in range(1, function_count+1):
    Map_lambda_logs_dict['f'+str(i)] += [lambda_report_log_to_dict(item) for item in query_lambda_results[i-1]['results']]
    for item in Map_lambda_logs_dict['f'+str(i)]:
        item['Function']='f'+str(i)

In [142]:
query_lambda = []
for function in function_name_list:
    query_lambda.append(logclient.start_query(
        logGroupName='/aws/lambda/{}'.format(function),
        queryString="fields @timestamp, @message| filter @message like 'REPORT'| sort @timestamp asc",
        startTime=process_time_string('2022-01-05 01:45:59.467') - 5,
        endTime=process_time_string('2022-01-05 04:35:14.979') - 5,
        limit=10000
        ))
    time.sleep(4)
time.sleep(10)
query_lambda_results = []
for q in query_lambda:
    query_lambda_results.append(logclient.get_query_results(
        queryId=q['queryId']
    ))
    time.sleep(4)
with open('query_lambda_results_part3.pickle', 'wb') as f:
    f.write(pickle.dumps(query_lambda_results))
for i in range(1, function_count+1):
    Map_lambda_logs_dict['f'+str(i)] += [lambda_report_log_to_dict(item) for item in query_lambda_results[i-1]['results']]
    for item in Map_lambda_logs_dict['f'+str(i)]:
        item['Function']='f'+str(i)

In [143]:
query_lambda = []
for function in function_name_list:
    query_lambda.append(logclient.start_query(
        logGroupName='/aws/lambda/{}'.format(function),
        queryString="fields @timestamp, @message| filter @message like 'REPORT'| sort @timestamp asc",
        startTime=process_time_string('2022-01-05 04:35:14.979') - 5,
        endTime=process_time_string('2022-01-05 07:24:26.929') - 5,
        limit=10000
        ))
    time.sleep(4)
time.sleep(10)
query_lambda_results = []
for q in query_lambda:
    query_lambda_results.append(logclient.get_query_results(
        queryId=q['queryId']
    ))
    time.sleep(4)
with open('query_lambda_results_part4.pickle', 'wb') as f:
    f.write(pickle.dumps(query_lambda_results))
for i in range(1, function_count+1):
    Map_lambda_logs_dict['f'+str(i)] += [lambda_report_log_to_dict(item) for item in query_lambda_results[i-1]['results']]
    for item in Map_lambda_logs_dict['f'+str(i)]:
        item['Function']='f'+str(i)

In [144]:
query_lambda = []
for function in function_name_list:
    query_lambda.append(logclient.start_query(
        logGroupName='/aws/lambda/{}'.format(function),
        queryString="fields @timestamp, @message| filter @message like 'REPORT'| sort @timestamp asc",
        startTime=process_time_string('2022-01-05 07:24:26.929') - 5,
        endTime=process_time_string('2022-01-05 10:13:44.352') + 5,
        limit=10000
        ))
    time.sleep(4)
time.sleep(10)
query_lambda_results = []
for q in query_lambda:
    query_lambda_results.append(logclient.get_query_results(
        queryId=q['queryId']
    ))
    time.sleep(4)
with open('query_lambda_results_part5.pickle', 'wb') as f:
    f.write(pickle.dumps(query_lambda_results))
for i in range(1, function_count+1):
    Map_lambda_logs_dict['f'+str(i)] += [lambda_report_log_to_dict(item) for item in query_lambda_results[i-1]['results']]
    for item in Map_lambda_logs_dict['f'+str(i)]:
        item['Function']='f'+str(i)

In [145]:
with open('Map_lambda_logs_dict.pickle', 'wb') as f:
    f.write(pickle.dumps(Map_lambda_logs_dict))

#### Convert Logs into DataFrame and Save as CSV

In [147]:
Map_lambda_logs=pd.DataFrame()
for i in range(1, function_count+1):
    Map_lambda_logs = Map_lambda_logs.append(pd.DataFrame(Map_lambda_logs_dict['f'+str(i)]))
Map_lambda_logs.index=range(Map_lambda_logs.shape[0])
Map_lambda_logs=Map_lambda_logs[['Function', 'Memory_Size', 'Max_Memory_Used', 'Duration', 'Billed_Duration', 'UTC_Timestamp', 'RequestId']]
Map_lambda_logs.to_csv('Map_lambda_logs.csv',index=False)

In [148]:
Map_lambda_logs = pd.read_csv('Map_lambda_logs.csv', low_memory=False)
Map_lambda_logs.columns = ['Function', 'Memory_Size', 'Max_Memory_Used', 'Duration', 'Billed_Duration', 'UTCTimestamp', 'RequestId']

In [149]:
Map_lambda_logs.head()

,Function,Memory_Size,Max_Memory_Used,Duration,Billed_Duration,UTCTimestamp,RequestId
0,f1,1280,36,277.59,278,1.641327e+09,230c5bae-1ec3-4589-9f6c-c96ca9d1f7d6
1,f1,1280,36,277.63,278,1.641327e+09,d891bf21-8f35-4da2-861d-568ed8c2fcf3
2,f1,1280,36,277.07,278,1.641327e+09,54f47eef-1471-449a-8132-652c37559431
3,f1,1280,36,268.12,269,1.641327e+09,2466b370-b620-4ab1-a390-529dfbcc0ee1
4,f1,1280,36,276.60,277,1.641327e+09,98889bb3-0027-454b-b41d-de50e435bff2


In [150]:
for i in range(1, function_count+1):
    print(f"f{i}", Map_lambda_logs.query(f"Function == 'f{i}'").shape[0], Map_lambda_logs.query(f"Function == 'f{i}'")['Duration'].mean())

f1 49990 279.33476795359076
f2 49990 207.9034602920584
f3 49990 686.8969289857972
f4 49990 305.89091798359675


In [151]:
def calculate_cost(rt: float, mem: float, pmms: float = 0.0000166667/1024/1000, ppi: float = 0.0000002) -> float:
    return math.ceil(rt) * mem * pmms + ppi

def adjacent_values(vals, q1, q3):
    upper_adjacent_value = q3 + (q3 - q1) * 1.5
    upper_adjacent_value = np.clip(upper_adjacent_value, q3, vals[-1])

    lower_adjacent_value = q1 - (q3 - q1) * 1.5
    lower_adjacent_value = np.clip(lower_adjacent_value, vals[0], q1)
    return lower_adjacent_value, upper_adjacent_value

# End-to-end RT Reported by AWS

In [201]:
Map_duration = pd.DataFrame((Map_sfn_logs['End'] - Map_sfn_logs['Start'])*1000, columns=['Duration'])
Map_duration.to_csv('Map_duration_aws.csv', index=False)
Map_duration = pd.read_csv('Map_duration_aws.csv', low_memory=False)

In [202]:
print('Number of Executions: ', len(Map_duration['Duration']))
Map_avg_duration_aws = np.mean(Map_duration['Duration'])
Map_mid_duration_aws = np.median(Map_duration['Duration'])
Map_percentile10_aws = np.percentile(Map_duration['Duration'], 10)
Map_percentile90_aws = np.percentile(Map_duration['Duration'], 90)
print('Average Duration Reported by AWS: ', Map_avg_duration_aws, 'ms')
print('Median Duration Reported by AWS: ', Map_mid_duration_aws, 'ms')
print('10-th percentile of Duration Reported by AWS: ', Map_percentile10_aws, 'ms')
print('90-th percentile Duration Reported by AWS: ', Map_percentile90_aws, 'ms')
print('Standard Deviation of Duration Reported by AWS: ', np.std(Map_duration['Duration']), 'ms')

Number of Executions:  5000
Average Duration Reported by AWS:  1693.0984003067017 ms
Median Duration Reported by AWS:  1677.0000457763672 ms
10-th percentile of Duration Reported by AWS:  1615.0000095367432 ms
90-th percentile Duration Reported by AWS:  1746.000051498413 ms
Standard Deviation of Duration Reported by AWS:  142.55836266757547 ms


# Cost Reported by AWS

In [203]:
Map_cost_aws = pd.DataFrame(pd.Series(cost_list), columns=['Cost'])
Map_cost_aws.to_csv('Map_cost_aws.csv', index=False)
Map_duration_model = pd.DataFrame(pd.Series(ert), columns=['Duration'])
Map_duration_model.to_csv('Map_duration_model.csv', index=False)
Map_cost_model = pd.DataFrame(pd.Series(ec), columns=['Cost'])
Map_cost_model.to_csv('Map_cost_model.csv', index=False)Map_sfn_logs.head()

,Start,End
0,1.641327e+09,1.641327e+09
1,1.641327e+09,1.641327e+09
2,1.641327e+09,1.641327e+09
3,1.641327e+09,1.641327e+09
4,1.641327e+09,1.641327e+09


In [155]:
Map_lambda_logs.head()

,Function,Memory_Size,Max_Memory_Used,Duration,Billed_Duration,UTCTimestamp,RequestId
0,f1,1280,36,277.59,278,1.641327e+09,230c5bae-1ec3-4589-9f6c-c96ca9d1f7d6
1,f1,1280,36,277.63,278,1.641327e+09,d891bf21-8f35-4da2-861d-568ed8c2fcf3
2,f1,1280,36,277.07,278,1.641327e+09,54f47eef-1471-449a-8132-652c37559431
3,f1,1280,36,268.12,269,1.641327e+09,2466b370-b620-4ab1-a390-529dfbcc0ee1
4,f1,1280,36,276.60,277,1.641327e+09,98889bb3-0027-454b-b41d-de50e435bff2


In [158]:
cost_list = []
for index, row in Map_sfn_logs.iterrows():
    cost = 0
    app_start = row['Start'] - 2
    app_end = row['End'] + 2
    lambda_logs = Map_lambda_logs.query(f"""UTCTimestamp>{app_start} and UTCTimestamp<{app_end}""")
    for i, r in lambda_logs.iterrows():
        memory_size = r['Memory_Size']
        duration = r['Duration']
        cost += calculate_cost(rt=duration, mem=memory_size) * 1000000
    cost_list.append(cost)

In [159]:
Map_avg_cost_aws = np.mean(cost_list)
Map_mid_cost_aws = np.median(cost_list)
Map_percentile10_cost_aws = np.percentile(cost_list, 10)
Map_percentile90_cost_aws = np.percentile(cost_list, 90)
Map_std_cost_aws = np.std(cost_list)
print('Average Cost Reported by AWS: ', Map_avg_cost_aws, 'USD')
print('Median Cost Reported by AWS: ', Map_mid_cost_aws, 'USD')
print('10-th percentile of Cost Reported by AWS: ', Map_percentile10_cost_aws, 'USD')
print('90-th percentile Cost Reported by AWS: ', Map_percentile90_cost_aws, 'USD')
print('Standard Deviation of Cost Reported by AWS: ', Map_std_cost_aws, 'USD')

Average Cost Reported by AWS:  322.729971961885 USD
Median Cost Reported by AWS:  322.76156702187507 USD
10-th percentile of Cost Reported by AWS:  314.931343028125 USD
90-th percentile Cost Reported by AWS:  330.60720771312504 USD
Standard Deviation of Cost Reported by AWS:  8.041376464650796 USD


# End-to-end RT and Cost Derived from the Modeling Algorithm

In [160]:
import sys
sys.path.append('../../')
from slappsim.Structures import *
from slappsim.Function import *
from slappsim.PetriApp import *
from slappsim.States import *

In [180]:
Structures_lambda_logs = pd.read_csv('../structures/Structures_lambda_logs.csv', low_memory=False)
Structures_lambda_logs.columns = ['Function', 'Memory_Size', 'Max_Memory_Used', 'Duration', 'Billed_Duration',
                              'UTCTimestamp', 'RequestId']
scheduling_overhead = pd.read_csv('../sfn-delay/Scheduling_Overhead.csv')
scheduling_overhead = np.array(scheduling_overhead['scheduling_overhead'].to_list())
function_execution_delay = pd.read_csv('../sfn-delay/Function_Execution_Delay.csv')
function_execution_delay = np.array(function_execution_delay['Duration'].to_list())

In [198]:
rs = np.random.RandomState(64)
random.seed(64)

## Define the application

In [199]:
f1_rt = np.array(Structures_lambda_logs.query(f"Function=='f1'")['Duration'].to_list()[500:9501])
f1_pp_fun = partial(rs.choice, a=f1_rt)
f1 = Function(pf_fun=f1_pp_fun, mem=mem_config_list['f1'], name='f1')
f2_rt = np.array(Structures_lambda_logs.query(f"Function=='f2'")['Duration'].to_list()[500:9501])
f2_pp_fun = partial(rs.choice, a=f2_rt)
f2 = Function(pf_fun=f2_pp_fun, mem=mem_config_list['f2'], name='f2')
f3_rt = np.array(Structures_lambda_logs.query(f"Function=='f3'")['Duration'].to_list()[500:9501])
f3_pp_fun = partial(rs.choice, a=f3_rt)
f3 = Function(pf_fun=f3_pp_fun, mem=mem_config_list['f3'], name='f3')
f4_rt = np.array(Structures_lambda_logs.query(f"Function=='f4'")['Duration'].to_list()[500:9501])
f4_pp_fun = partial(rs.choice, a=f4_rt)
f4 = Function(pf_fun=f4_pp_fun, mem=mem_config_list['f4'], name='f4')
sfn_scheduling_overhead_fun = partial(rs.choice, a=scheduling_overhead)
function_execution_delay_fun = partial(rs.choice, a=function_execution_delay)
delays = {'FunctionExecution': function_execution_delay_fun, 'SchedulingOverhead': sfn_scheduling_overhead_fun}
start = Start()
end = End()
sequence1 = Sequence(actions=[f1, f2, f3, f4])
map1 = Map(sequence=sequence1, iterations=10)
structures = [map1, sequence1]
i1 = InArc(place=start)
o1 = OutArc(place=map1.structure_start)
t1 = Transition(in_arcs=[i1], out_arcs=[o1])
i2 = InArc(place=map1.structure_end)
o2 = OutArc(place=end)
t2 = Transition(in_arcs=[i2], out_arcs=[o2])
transitions = [t1, t2]
transitions += map1.transitions
AppMap = PetriApp(transitions=transitions,
                  functions=[f1, f2, f3, f4],
                  structures=structures,
                  delays=delays)

## Run the modeling algorithm

In [200]:
ert = []
ec = []
for i in range(100000):
    rt, c, s, logs = AppMap.execute()
    ert.append(rt)
    ec.append(c * 1000000)
    AppMap.reset()
Map_avg_cost_mdl = np.mean(ec)
Map_mid_cost_mdl = np.median(ec)
Map_percentile10_cost_mdl = np.percentile(ec, 10)
Map_percentile90_cost_mdl = np.percentile(ec, 90)
Map_std_cost_mdl = np.std(ec)
Map_avg_ert_mdl = np.mean(ert)
Map_mid_ert_mdl = np.median(ert)
Map_percentile10_ert_mdl = np.percentile(ert, 10)
Map_percentile90_ert_mdl = np.percentile(ert, 90)
Map_std_ert_mdl = np.std(ert)
print('Average Duration Reported by Algorithm: ', Map_avg_ert_mdl, 'ms')
print('Median Duration Reported by Algorithm: ', Map_mid_ert_mdl, 'ms')
print('10-th percentile of Duration Reported by Algorithm: ', Map_percentile10_ert_mdl, 'ms')
print('90-th percentile Duration Reported by Algorithm: ', Map_percentile90_ert_mdl, 'ms')
print('Standard Deviation of Duration Reported by Algorithm: ', Map_std_ert_mdl, 'ms')
print('Average Cost Reported by Algorithm: ', Map_avg_cost_mdl, 'USD')
print('Median Cost Reported by Algorithm: ', Map_mid_cost_mdl, 'USD')
print('10-th percentile of Cost Reported by Algorithm: ', Map_percentile10_cost_mdl, 'USD')
print('90-th percentile Cost Reported by Algorithm: ', Map_percentile90_cost_mdl, 'USD')
print('Standard Deviation of Cost Reported by Algorithm: ', Map_std_cost_mdl, 'USD')

Average Duration Reported by Algorithm:  1717.6143628537095 ms
Median Duration Reported by Algorithm:  1683.3201239776608 ms
10-th percentile of Duration Reported by Algorithm:  1629.9186782302859 ms
90-th percentile Duration Reported by Algorithm:  1783.8101707077028 ms
Standard Deviation of Duration Reported by Algorithm:  436.6404480676971 ms
Average Cost Reported by Algorithm:  329.6629424495982 USD
Median Cost Reported by Algorithm:  329.36418439375007 USD
10-th percentile of Cost Reported by Algorithm:  322.49219148312505 USD
90-th percentile Cost Reported by Algorithm:  336.29961493124995 USD
Standard Deviation of Cost Reported by Algorithm:  9.41892518378295 USD


In [204]:
Map_cost_aws = pd.DataFrame(pd.Series(cost_list), columns=['Cost'])
Map_cost_aws.to_csv('Map_cost_aws.csv', index=False)
Map_duration_model = pd.DataFrame(pd.Series(ert), columns=['Duration'])
Map_duration_model.to_csv('Map_duration_model.csv', index=False)
Map_cost_model = pd.DataFrame(pd.Series(ec), columns=['Cost'])
Map_cost_model.to_csv('Map_cost_model.csv', index=False)